# Практическое задание 1

# Ранжирование вопросов StackOverflow с помощью векторных представлений слов

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

В этом задании вы научитесь вычислять близость текстов и применить этот метод для поиска похожих вопросов на [StackOverflow](https://stackoverflow.com).

### Используемые библиотеки

В данном задании потребуются следующие библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...).
- [Numpy](http://www.numpy.org) — библиотека для научных вычислений.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с многими реализованными алгоритмами машинного обучения для анализа данных.
- [Nltk](http://www.nltk.org) — инструмент для работы с естественными языками.
- [Pytorch](https://pytorch.org/) — инструмент для обучения нейросетей.


### Данные

Данные лежат в архиве `StackOverflowData.zip`, который состоит из:
- `train.tsv` - обучающая выборка. В каждой строке через табуляцию записаны дублирующие друг друга предложения;
- `test.tsv` - тестовая выборка. В каждой строке через табуляцию записаны: *<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...*

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_)

#### Тесты

In [1]:
from tests import TaskTests

task_tests = TaskTests.from_json(path='test_gt.json')

In [2]:
import torch

In [4]:
! nvidia-smi

Mon Sep 23 18:18:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:18:00.0 Off |                    0 |
| N/A   25C    P0              70W / 700W |      3MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### Вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов. Используйте [модель эмбеддингов](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit), которая была обучена с помощью пакета word2vec на данных Google News (100 миллиардов слов). Модель содержит 300-мерные вектора для 3 миллионов слов и фраз. Вы можете скачать их, запустив блок кода ниже.

In [5]:
# from download_utils import download_google_vectors


# download_google_vectors(target_dir='.')

## Часть 1. Предобученные векторные представления слов (2 балла)

Скачайте предобученные вектора и загрузите их с помощью функции [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) библиотеки Gensim с параметром *binary=True*. Если суммарный размер векторов больше, чем доступная память, то вы можете загрузите только часть векторов, указав параметр *limit* (рекомендуемое значение: 500000).

In [10]:
import gensim


wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000,
)

### Как пользоваться этими векторами?

Как только вы загрузите векторные представления слов в память, убедитесь, что имеете к ним доступ. Сначала вы можете проверить, содержится ли какое-то слово в загруженных эмбедингах:

    'word' in wv_embeddings

Затем, чтобы получить соответствующий вектор, вы можете использовать оператор доступа по ключу:

    wv_embeddings['word']

### Проверим, корректны ли векторные представления

Чтобы предотвратить возможные ошибки во время первого этапа, можно проверить, что загруженные вектора корректны. Для этого проверьте три пункта:
1. Используя метод `.most_similar(positive=..., negative=...)`, найти слово, похожее на `woman`, `king` и непохожее на `man`.
2. Используя метод `.doesnt_match(...)`, найти "белую ворону" в списке `['breakfast, 'dinner', 'lunch', 'cereal']`.
3. Используя метод `.most_similar_to_given(word, [...])`, найти наиболее похожее на `music` слово из списка `['water', 'sound', 'backpack', 'mouse']`.

Прокомментируйте полученные результаты: считаете ли вы их верными и почему.

In [7]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [8]:
'word' in wv_embeddings

True

In [9]:
wv_embeddings['word'].shape

(300,)

In [10]:
wv_embeddings.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454),
 ('royal_palace', 0.5087166428565979)]

In [11]:
wv_embeddings.doesnt_match(['breakfast', 'dinner', 'lunch', 'cereal'])

'cereal'

In [12]:
wv_embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])

'sound'

### Ранжирование вопросов StackOverflow

Давайте посмотрим на данные, которые мы будем использовать в рамках задания. Выборка уже разбита на обучающую и тестовую. Все файлы используют табуляцию в качестве разделителя, но они имеют разный формат:

- *обучающая* выборка (train.tsv) содержит похожие друг на друга предложения в каждой строке;
- *тестовая* выборка (validation.tsv) содержит в каждой строке: *вопрос, похожий вопрос, отрицательный пример 1, отрицательный пример 2, ...*

Считайте тестовую (валидационную) выборку. Ответьте на следующие вопросы:
1. Сколько пар-дубликатов предоставлено в выборке?
2. Сколько в среднем на каждую пару предоставлено отрицательных примеров?

In [11]:
import tqdm


def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

train = read_corpus('train.tsv')
validation = read_corpus('validation.tsv')

In [15]:
import numpy as np

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [101]:
len(train), train[0]

(1000000,
 ['converting string to list',
  'Convert Google results object (pure js) to Python object'])

In [28]:
len(validation), validation[0][0], validation[0][1], validation[0][2]

(3760,
 'How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js')

In [29]:
num_samples = len(validation)
amount_of_negatives_per_sample = 0
for example in validation:
    amount_of_negatives_per_sample += len(example) - 2
amount_of_negatives_per_sample /= num_samples
num_samples, amount_of_negatives_per_sample

(3760, 998.8106382978723)

In [30]:
task_tests.test_validation_corpus(
    num_samples,
    amount_of_negatives_per_sample
)

### Векторные представления текста

Чтобы перейти от отдельных слов к векторным представлениям вопросов, предлагается подсчитать **среднее** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектоора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.



In [3]:
import numpy as np


class Embedder:
    
    def __init__(self, embeddings, dim):
        """
            embeddings: word2vec эмбеддинги
            dim: размерность word2vec эмбеддингов. Нужна для задания нулего вектора для пустых вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.embeddings = embeddings
        self.dim = dim
        
    def __call__(self, text, normalize=False):
        """
            Принимает на вход текст и преобразует его в вектор.
            
            text: строка с вопросом
            normalize: при True нужно перед возвращением нормализовать вектор
            
            returns: вектор вопроса
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        words = text.split()
        sum_embed = np.zeros((self.dim))
        cnt = 0
        for word in words:
            if word in self.embeddings:
                sum_embed += self.embeddings[word]
                cnt += 1
        if cnt > 0:
            sum_embed /= cnt
            if normalize:
                sum_embed /= np.linalg.norm(sum_embed)
        return sum_embed

In [46]:
embedder = Embedder(wv_embeddings, dim=300)

In [47]:
task_tests.test_embedder(embedder)

Теперь у нас есть метод для создания векторного представления любого предложения. Оценим, как будет работать это решение.

### Оценка близости текстов

В качестве метрики схожести вопросов будем использовать косинусную близость.

В валидационном датасете для каждой пары вопросов-дубликатов у нас есть случайные отрицательные примеры. Для каждого триплета (вопрос, дубликат, отрицательные примеры) будет ранжировать с помощью нашей модели и косинусной близости дубликат и отрицательные примеры и смотреть на позицию дубликата.

#### Hits@K
Довольно простой и легко интерпретируемой метрикой будет количество корректных попаданий дубликата в top "выдачи" для какого-то *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)],$$
где $q_i$ - $i$-ый вопрос, $dup_i$ - его дубликат, $topK(q_i)$ - первые *K* элементов в ранжированном списке, который выдает наша модель.

#### Пример оценок

Пусть $N = 1$, вопрос $q_1$ это "Что такое python", а его дубликат $dup_1$ это "Что такое язык python". Пусть модель выдала следующий ранжированный список кандидатов:

1. *"Как узнать с++"*
2. *"Что такое язык python"*
3. *"Хочу учить Java"*
4. *"Не понимаю Tensorflow"*

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [dup_1 \in top1(q_1)] = 0$
- [K = 4] $\text{Hits@4} =  [dup_1 \in top4(q_1)] = 1$

#### Подсчет метрики Hits@k сразу для нескольких k

Чтобы посчитать метрику для нескольких k, не нужно повторно ранжировать нашей моделью вопросы для одного и того же сэмпла. Достаточно посчитать для сэмпла количество **сложных негативов** - отрицательных примеров, оказавшихся в выдаче выше, чем дубликат. Тогда
$$Hits@k = \begin{cases}
    1, & N < k \\
    0, & иначе
   \end{cases},$$
где **N** - количество сложных негативов.

Реализуйте подсчет Hits@k для произвольного набора значений k и заданной валидационной выборки, используя предложенный шаблон.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
        
    
class Scorer:
    
    def __init__(self, k, embedder):
        """
            k: список значений k, для которых нужно посчитать hits@k
            embedder: объект класса Embedder, умеющий преобразовать текст в вектор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.k = k
        self.embedder = embedder
        
    def _get_hard_negatives(self, q, pos, negs):
        """
            q: текст вопроса
            pos: текст дубликата
            negs: список из текстов случайных вопросов
            
            result: количество сложных отрицательных примеров, оказавшихся выше положительного
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        q_embedding = self.embedder(q).reshape(1, -1)
        pos_embedding = self.embedder(pos).reshape(1, -1)
        pos_sim = cosine_similarity(q_embedding, pos_embedding)
        cnt = 0
        for neg in negs:
            neg_embedding = self.embedder(neg).reshape(1, -1)
            neg_sim = cosine_similarity(q_embedding, neg_embedding)
            if neg_sim > pos_sim:
                cnt += 1
        return cnt
    
    def __call__(self, samples, verbose=False):
        """
            samples: список из списков вида [q, pos, neg1, neg2, ...]. Наша валидационная выборка
            verbose: выводить progressbar подсчета метрики с помощью tqdm
            
            result: словарь вида {k: hits@k}
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        n = np.array([])
        for sample in tqdm.tqdm(samples, disable = not verbose):
            n = np.append(n, self._get_hard_negatives(sample[0], sample[1], sample[2:]))
        ans = {}
        for k in self.k:
            ans[k] = (n < k).mean()
        return ans

In [71]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(validation, verbose=True)

100%|██████████| 3760/3760 [12:15<00:00,  5.11it/s]


In [73]:
hits

{1: 0.21436170212765956,
 5: 0.3125,
 10: 0.36622340425531913,
 100: 0.5640957446808511,
 500: 0.8207446808510638,
 1000: 1.0}

In [72]:
task_tests.test_scorer(hits)

### Предобработка текста

Как вы могли заметить, мы имеем дело с сырыми данными. Это означает, что там присутствует много опечаток, спецсимволов и заглавных букв. В нашем случае это все может привести к ситуации, когда для данных токенов нет предобученных векторов. Поэтому необходима предобработка.

Вам требуется:
- Перевести символы в нижний регистр;
- Заменить символы пунктуации и всевозможные плохие символы на пробелы;
- Удалить стопслова.
- Удалить слова с длиной меньше трех букв

Реализуйте предобработку текста, используя предложенный шаблон.

In [5]:
import re
    
    
class TextPreprocessor:
    
    def __init__(self, characters, min_word_length=0, stopwords=None):
        """
            characters: список плохих символов
            min_word_length: минимальная допустимая длина для слов
            stopwords: множество фоновых слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.characters = characters
        self.min_word_length = min_word_length
        self.stopwords = stopwords

    def __call__(self, text):
        """
            text: текст для обработки
            
            returns: обработанный текст
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        bad_chars_regex = '[' + re.escape(''.join(self.characters)) + ']'
        text = re.sub(bad_chars_regex, ' ', text.lower())
        words = text.split()
        return ' '.join([word for word in words if len(word) >= self.min_word_length and word not in self.stopwords])

In [79]:
task_tests.test_text_preprocessor(TextPreprocessor)

Множество фоновых слов можно взять из **nltk** с помощью `nltk.corpus.stopwords.words`, выкидываемые плохие символы и пунктуацию следует подобрать самостоятельно.

Обработайте текст и продемонстрируйте улучшение качества:

In [6]:
import nltk
from nltk.corpus import stopwords
import string


nltk.download('stopwords')
    

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

stop_words = set(stopwords.words('english'))
characters = list(string.punctuation)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
text_preprocessor = TextPreprocessor(characters, 3, stop_words)
train_preproc = [[text_preprocessor(text) for text in sample] for sample in tqdm.tqdm(train)]
val_preproc = [[text_preprocessor(text) for text in sample] for sample in tqdm.tqdm(validation)]

100%|██████████| 3760/3760 [00:12<00:00, 300.16it/s]


In [88]:
scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(val_preproc, verbose=True)

100%|██████████| 3760/3760 [11:58<00:00,  5.24it/s]


In [89]:
hits

{1: 0.3406914893617021,
 5: 0.47819148936170214,
 10: 0.5289893617021276,
 100: 0.7007978723404256,
 500: 0.8555851063829787,
 1000: 1.0}

Одним из критериев получения полных баллов является значение **hits@500** $\geqslant 0.82$ до предобработки текста и $\geqslant 0.85$ после предобработки.

## Часть 2. Представления для неизвестных слов. (4 балла)

Для того, что получить представления для неизвестного слова, воспользуемся следующим подходом:
    
1. Будем восстанавливать эмбеддинг неизвестного слова как сумму эмбеддингов буквенных триграмм. Например, слово where должно представляться суммой триграмм #wh, whe, her, ere, re#

2. В качестве обучающих данных будем использовать слова, для которых есть эмбеддинг в модели. Будем обучать эмбеддинги триграмм по выборке эмбеддингов с помощью функционала MSE:

$$L = \sum_{w \in W_{known}}\| f_{\theta}(w) - v_w \|^2 \to \min_{\theta}$$

где:

* $W_{known}$ — множество известных модели слов
* $f_{\theta}(w)$ — сумма эмбеддингов триграмм слова $w$
* $v_w$ — эмбеддинг слова $w$
* $\theta$ — веса эмбеддингов триграмм

### Создание триграммного токенизатора

Для начала, нам нужно:
1. Пройтись по известным в word2vec словам и составить множество триграмм, для которых будем обучать векторы
2. Составить маппинг из триграмм в индексы
3. Реализовать преобразование произвольного слова в список триграмм
4. Реализовать преобразование произвольного слова в список индексов триграмм

Для реализации всех этих пунктов предлагается использовать шаблон, приведенный ниже.

In [291]:
class TrigramTokenizer:
    
    def __init__(self, words):
        """
            Формируем множество всевозможных триграмм, встречающихся в словах из words.
            Делаем маппинг триграмм в индексы.
            
            words: список слов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.words = words
        trigrams = set()
        for word in words:
            trigrams.update(self._get_trigrams(word))
        self.trigrams = {trigram: idx for idx, trigram in enumerate(trigrams)}
        
    @property
    def vocab_size(self):
        """
            returns: колчиество триграмм, для которых мы завели индекс.
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.trigrams)
    
    @staticmethod
    def _get_trigrams(word):
        """
            word: слово
            
            returns: список триграмм для word
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        trigrams = [word[i : i+3] for i in range(len(word) - 2)]
        return ['#' + word[:2]] + trigrams + [ word[-2:] + '#']
        
    def __call__(self, word):
        """
            word: слово
            
            returns: список индексов триграмм для слова word, которые нашлись в маппинге
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        trigrams = self._get_trigrams(word)
        return [self.trigrams[trigram] for trigram in trigrams if trigram in self.trigrams]

In [95]:
task_tests.test_trigram_tokenizer(TrigramTokenizer)

Для создания токенизатора используйте обработанный с помощью TextProcessor текст. 

**Важно:** в токенизатор нужно подавать только слова, известные word2vec'у.

In [103]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

val_preproc_known_words = []
for sample in tqdm.tqdm(val_preproc):
    for text in sample:
        for word in text.split():
            if word not in val_preproc_known_words and word in wv_embeddings:
                val_preproc_known_words.append(word)


100%|██████████| 3760/3760 [03:46<00:00, 16.61it/s]


In [292]:
tri_tokenizer = TrigramTokenizer(val_preproc_known_words)

### Создание датасета с w2v векторами и списками индексов триграмм

Мы будем обучать триграммную модель в нейросетевом фреймворке pytorch. Для этого нам нужно создать свой датасет.

Он должен:
1. Принимать список слов, word2vec и уже созданный триграммный токенизатор.
2. Выдавать пары вида (эмбеддинг для слова из word2vec, список индексов триграмм для этого слова)

Реализовать датасет нужно в шаблоне, приведенном ниже.

In [119]:
from torch.utils.data import Dataset


class TrainTrigramDataset(Dataset):
    
    def __init__(self, vocab, w2v_embeddings, tri_tokenizer):
        """
            Формируем выборку для обучения триграммной модели.
            ЗАРАНЕЕ считаем маппинг в список индексов для всех известных в word2vec слов.
            
            vocab: список слов
            w2v_embeddings: no comments
            tri_tokenizer: токенизатор триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab
        self.w2v_embeddings = w2v_embeddings
        self.tri_tokenizer = tri_tokenizer
        
        self.word_to_idx = {word: idx for idx, word in enumerate(vocab)}
                
    def __len__(self):
        """
            returns: возвращает количество слов, вошедших в маппинг (размер словаря)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.vocab)
    
    def __getitem__(self, idx):
        """
            returns: w2v эмбеддинг для idx-го слова в датасете, список соответствующих ему триграмм (тензоры)
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        word = self.vocab[idx]
        return self.w2v_embeddings[word], self.tri_tokenizer(word)
    
w2v_vocab = val_preproc_known_words
ds = TrainTrigramDataset(w2v_vocab, wv_embeddings, tri_tokenizer)

In [120]:
task_tests.test_dataset(ds, w2v_vocab, wv_embeddings, tri_tokenizer)

### Создание DataLoader'а и Collator'а

Нас интересуют в первую очередь четыре параметра при создании DataLoader:
1. Датасет. Реализует интерфейс массива - можно узнать длину и получить элемент с индексом, меньшим длины.
2. batch_size. Задает размера батча (количества сэмплов, идущих одновременно в модель).
3. shuffle. При shuffle == True каждую эпоху при итерировании по даталоадеру мы будем получать сэмплы в произвольном порядке.
4. collate_fn. Этот параметр позволяет задать кастомную логику "склеивания" сэмплов из датасета в батч.

В качестве модели мы будем использовать слой **torch.nn.EmbeddingBag**. Он принимает на вход список индексов и список сдвигов, начинающийся с нуля.

Нужно наш список списков индексов триграмм превратить в соответствующий формат, преобразовать векторы слов и два списка (индексов и сдвигов) в pytorch тензоры (torch.tensor).

Реализуйте следующую функцию:

In [166]:
import torch
from torch.utils.data import DataLoader


def collate_fn(batch):
    """
        batch: список из элементов датасета, e.g. [ds[i] for i in [2, 3, 1, 15]]
        
        returns: w2v эмбеддинги, индексы триграмм, сдвиги для триграмм
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    w2v = []
    trigram_idx = []
    trigram_offsets = []

    for item in batch:
        trigram_offsets.append(len(trigram_idx))
        w2v.append(item[0])
        trigram_idx += item[1]
    return torch.tensor(w2v), torch.tensor(trigram_idx), torch.tensor(trigram_offsets)


In [167]:
task_tests.test_dataloader(ds, collate_fn, embedding_dim=300)

### Создание модели

При создании модели мы обычно наследуемся от **torch.nn.Module** и создаем нужные нам слои как атрибуты объекта нашего класса.

В данном случае предлагается для формирования эмбеддингов использовать **torch.nn.EmbeddingBag**.

Реализуйте предложенный шаблон:

In [168]:
from torch import nn


class TrigramModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество триграмм, для которых обучаются эмбеддинги
            embedding_dim: размерность эмбеддингов триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        super(TrigramModel, self).__init__()
        self.num_embed = num_embeddings
        self.embed_dim = embedding_dim
        self.embed = nn.EmbeddingBag(num_embeddings, embedding_dim)
        
    @property
    def embedding_dim(self):
        """
            returns: размерность эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.embed_dim
    
    @property
    def num_embeddings(self):
        """
            returns: количество эмбеддингов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.num_embed
    
    def forward(self, trigrams, offsets):
        """
            trigrams: список индексов триграмм (тензор)
            offsets: список сдвигов (тензор)
            
            returns: эмбеддинги слов, составленные из триграмм
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return self.embed(trigrams, offsets)
    
model = TrigramModel(tri_tokenizer.vocab_size, embedding_dim=wv_embeddings.vector_size)

In [169]:
task_tests.test_trigram_model(model)

### Создание пайплайна обучения

Далее необходимо совместить все наработки в единый пайплайн обучения, добавив также критерий для оптимизации и оптимизатор. 

Предлагается:

1. В качестве оптимизатора использовать Adam (можно попробовать подобрать learning rate / weight decay)
2. В качестве критерия оптимизации взять nn.MSELoss (можно также закодить лосс самому)
3. Для даталоадера выбрать небольшой батч сайз (32, 64, 128, 256)
4. Десяти эпох должно быть достаточно для хорошего качества

Реализуйте предложенный шаблон.

In [178]:
import time

    
class Trainer:
    
    def __init__(self, model, criterion, optimizer):
        """
            model: триграммная модель
            criterion: функционал ошибки, принимает на вход w2v эмбеддинги и триграммные эмбеддинги
            optimizer: оптимизатор для модели
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def _train_step(self, dataloader):
        """
            Делаем один проход по даталоадеру, с бэкпропом
            
            dataloader: даталоадер с тренировочными данными
            
            returns: лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model.train()
        total_loss = 0.0
    
        for w2v_true, trigrams, offsets in dataloader:
            self.optimizer.zero_grad()
            w2v_pred = self.model(trigrams, offsets)
            loss = self.criterion(w2v_pred, w2v_true)
            loss.backward()  # Распространение ошибки назад
            self.optimizer.step()
    
            total_loss += loss.item()    
        return total_loss / len(dataloader)
    
    def train(self, dataloader, n_epochs, verbose=False):
        """
            dataloader: тренировочный даталоадер
            n_epochs: количество эпох
            verbose: выводить лосс каждую эпоху или нет
            
            returns: список лоссов
        """
        start = time.time()
        losses = []
        for epoch in range(n_epochs):
            loss = self._train_step(dataloader)
            losses.append(loss)
            if verbose:
                print(f'epoch: {epoch + 1:>2}, loss: {loss:.4f}, time: {time.time() - start:.4f}')
        return losses


###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [179]:
from torch.utils.data import DataLoader

In [180]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
dataloader = DataLoader(ds, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [181]:
trainer = Trainer(model, criterion, optimizer)
losses = trainer.train(dataloader, n_epochs=30, verbose=True)

epoch:  1, loss: 0.1327, time: 0.5750
epoch:  2, loss: 0.1305, time: 1.1567
epoch:  3, loss: 0.1287, time: 1.7666
epoch:  4, loss: 0.1265, time: 2.3674
epoch:  5, loss: 0.1246, time: 2.9553
epoch:  6, loss: 0.1225, time: 3.5455
epoch:  7, loss: 0.1205, time: 4.1376
epoch:  8, loss: 0.1187, time: 4.7131
epoch:  9, loss: 0.1169, time: 5.2720
epoch: 10, loss: 0.1152, time: 5.8208
epoch: 11, loss: 0.1134, time: 6.3497
epoch: 12, loss: 0.1120, time: 6.8827
epoch: 13, loss: 0.1099, time: 7.4351
epoch: 14, loss: 0.1086, time: 8.0171
epoch: 15, loss: 0.1069, time: 8.5667
epoch: 16, loss: 0.1052, time: 9.1143
epoch: 17, loss: 0.1038, time: 9.6541
epoch: 18, loss: 0.1023, time: 10.2090
epoch: 19, loss: 0.1009, time: 10.7496
epoch: 20, loss: 0.0996, time: 11.2839
epoch: 21, loss: 0.0982, time: 11.8403
epoch: 22, loss: 0.0967, time: 12.3765
epoch: 23, loss: 0.0955, time: 12.9305
epoch: 24, loss: 0.0942, time: 13.4568
epoch: 25, loss: 0.0929, time: 14.0159
epoch: 26, loss: 0.0916, time: 14.5564
epo

### Получение векторов неизвестных слов. Инференс модели

Теперь, когда мы обучили модель, нам необходимо применить её для всех неизвестных слов, т.е. получить для них эмбеддинги.

Т.к. для этих слов у нас нет word2vec эмбеддингов, то dataset и collator для обучения не подходят для инференса. Необходимо реализовать датасет и коллатор для инференса по следующим шаблонам:

In [311]:
class InferenceTrigramDataset:
    
    def __init__(self, vocab, tri_tokenizer):
        """
            Датасет с неизвестными словами
            
            vocab: список слов
            tri_tokenizer: триграммный токенизатор
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab
        self.tri_tokenizer = tri_tokenizer
        
    def __len__(self):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.vocab)
    
    def __getitem__(self, idx):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        word = self.vocab[idx]
        return idx, self.tri_tokenizer(word)
    
    
def inference_collate_fn(trigrams):
    """
        trigrams: список списков индексов триграмм
        
        returns: список индексов, список сдвигов триграмм
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    trigram_idx = []
    trigram_offsets = []
    idx = []

    for (i, trigram) in trigrams:
        trigram_offsets.append(len(trigram_idx))
        trigram_idx += trigram
        idx += [i]
    return torch.tensor(idx), torch.tensor(trigram_idx), torch.tensor(trigram_offsets)

Теперь у нас есть всё необходимое, чтобы осуществить инференс. Не забудь перед инференсом перевести модель в режим эвала (**model.eval**), а также использовать контекстный менеджер **torch.no_grad**.

После инференса сформируйте словарь из известных в word2vec слов и их эмбеддингов, затем дополните его эмбеддингами для неизвестных слов, полученными после инференса.

In [ ]:
val_preproc_unknown_words = []
for sample in tqdm.tqdm(val_preproc):
    for text in sample:
        for word in text.split():
            if word not in val_preproc_unknown_words and word not in wv_embeddings:
                val_preproc_unknown_words.append(word)

tri_tokenizer_unknown = TrigramTokenizer(val_preproc_unknown_words)

 55%|█████▌    | 2077/3760 [08:07<06:38,  4.22it/s]

In [302]:
len(val_preproc_unknown_words)

9527

In [312]:
w2v_vocab_unknown = val_preproc_unknown_words
ds_unknown = InferenceTrigramDataset(w2v_vocab_unknown, tri_tokenizer)
dataloader_known = DataLoader(ds, batch_size=64, shuffle=False, collate_fn=collate_fn)
dataloader_unknown = DataLoader(ds_unknown, batch_size=64, shuffle=False,  collate_fn=inference_collate_fn)

In [313]:
trainer.model.eval()
unknown_pred = np.array([])
word2vec_embeddings = {}
with torch.no_grad():
    for idx, trigrams, offsets in dataloader_unknown:
        w2v_pred = model(trigrams, offsets)
        unknown_pred = np.append(unknown_pred, w2v_pred.numpy())

        for i, embed in zip(idx, w2v_pred):
            word = w2v_vocab_unknown[i]
            word2vec_embeddings[word] = embed.numpy()

In [315]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

for word in val_preproc_known_words:
    word2vec_embeddings[word] = wv_embeddings[word]

Используя **Scorer** и **Embedder**, получите новые значения метрик для валидации:

In [316]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

embedder = Embedder(word2vec_embeddings, dim=300)

scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(val_preproc, verbose=True)

100%|██████████| 3760/3760 [11:46<00:00,  5.32it/s]


In [317]:
hits

{1: 0.4332446808510638,
 5: 0.5694148936170212,
 10: 0.6172872340425531,
 100: 0.7656914893617022,
 500: 0.9071808510638298,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.89$.

## Часть 3. Обучение векторных представлений для целевой задачи. (4 баллов)

Предполагается, что в этой части используются TextPreprocessor, Embedder, Scorer из предыдущих частей.

Для обучения на целевую задачу нам понадобится обучающая выборка. Считайте её с диска, предобработайте текст вопросов

In [8]:
import tqdm

In [13]:
train = []
text_preprocessor = TextPreprocessor(characters, 3, stop_words)
for questions in tqdm.tqdm(read_corpus('train.tsv')):
    train.append([text_preprocessor(text) for text in questions])

100%|██████████| 1000000/1000000 [00:09<00:00, 107936.36it/s]


Необходимо создать **токенизатор для текста** - составить словарь и сделать маппинг из слов в индексы.

In [14]:
class TextTokenizer:
    
    def __init__(self, vocab):
        """
            vocab: множество слов, встретившихся в обучающей выборке
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.vocab = vocab

    @property
    def vocab_size(self):
        """
            returns: количество слов в словаре
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.vocab)
    
    def __call__(self, text):
        """
            text: текст
            
            returns: список индексов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        indices = []
        for word in text.split():
            indices.append(self.vocab[word])
        return indices

Составление словаря и токенизатора

In [15]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
train_preproc = [[text_preprocessor(text) for text in sample] for sample in tqdm.tqdm(train)]
val_preproc = [[text_preprocessor(text) for text in sample] for sample in tqdm.tqdm(validation)]

100%|██████████| 3760/3760 [00:12<00:00, 295.90it/s]


In [16]:
train_preproc_vocab = {}
idx = 0
for sample in tqdm.tqdm(train_preproc):
    for text in sample:
        for word in text.split():
            if word not in train_preproc_vocab:
                train_preproc_vocab[word] = idx
                idx += 1

100%|██████████| 1000000/1000000 [00:01<00:00, 640496.97it/s]


In [17]:
len(train_preproc_vocab)

131030

In [18]:
tokenizer = TextTokenizer(train_preproc_vocab)

Нам также понадобится **новый датасет для обучения**. Для применения метода NT-Exent нам не нужно "майнить негативы", поэтому датасет надо сформировать как массив из пар-дубликатов.

Так как данные в обучающей выборке содержат множества дубликатов (т.е. все дубликаты сгруппированы в списки), есть несколько способов сформировать итоговый датасет:
1. Оставить из каждого множества дубликатов какие-нибудь случайные два (или просто первые два вопроса)
2. Для первого вопроса в множестве взять все остальные как дубликаты (N вопросов-дубликатов - N-1 пара). Тогда мы увидим каждый вопрос хотя бы один раз при обучении
3. Составить всевозможные уникальные пары-дубликаты из этих множеств (т.е. первый вопрос и все остальные вопросы, второй вопрос и все остальные, кроме первого).

Каждый следующий способ, начиная с первого, раздувает выборку по размеру, но возможно дает прирост к качеству решения задачи.

Реализуйте выбранный вами подход, используя предолженный шаблон:

In [25]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [21]:
class QuestionDuplicatesDataset(Dataset):
    
    def __init__(self, question_pairs, tokenizer):
        """
            question_pairs: список из пар вопросов-дубликатов
            tokenizer: объект класса TextTokenizer
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.question_pairs = question_pairs
        self.tokenizer = tokenizer
            
    def __len__(self):
        """
            returns: количество пар-дубликатов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        return len(self.question_pairs)
    
    def __getitem__(self, idx):
        """ 
            returns: (вопрос, дубликат), idx-ю пару в датасете
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        question = self.tokenizer(self.question_pairs[idx][0])
        duplicate = self.tokenizer(self.question_pairs[idx][1])
        return question, duplicate

Также нужно подготовить **даталоадер** (а именно - коллатор для даталоадера) по аналогии со второй частью задания.

In [22]:
def get_ids_and_offsets(question):
    """
        questions: список из токенизированных вопросов
        
        returns: (ids, offsets), где ids - вытянутый список индексов слов в вопросах из батча, offsets - сдвиги
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    ids = []
    offsets = []
    for q in question:
        ids += q
        offsets.append(len(ids))
    return ids, offsets


def collate_fn(batch):
    """
        batch: список из пар токенизированных вопросов-дубликатов [(question, duplicate), ...]
        
        returns: (question_ids, question_offsets), (duplicate_ids, duplicate_offsets)
    """
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    questions, duplicates = [], []
    for (question, duplicate) in batch:
        questions.append(question)
        duplicates.append(duplicate)
    question_ids, question_offsets = get_ids_and_offsets(questions)
    duplicate_ids, duplicate_offsets = get_ids_and_offsets(duplicates)

    return (torch.tensor(question_ids), question_offsets), (torch.tensor(duplicate_ids), duplicate_offsets)

Поделите выборку на трейн и валидацию, используя train_test_split, затем **создайте датасеты и даталоадеры** для обучения и валидации. Сколько пар-дубликатов получилось в датасете для обучения?

In [23]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
from sklearn.model_selection import train_test_split

In [24]:
train_ds = QuestionDuplicatesDataset(train_preproc, tokenizer)
train_train, val_train = train_test_split(train_ds, test_size=0.2, shuffle=True)

In [26]:
train_dataloader = DataLoader(train_train, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_train, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [27]:
len(train_train), len(val_train)

(800000, 200000)

С помощью предложенного шаблона **задайте модель** для преобразования вопросов в векторы.

In [28]:
from torch import nn


class DssmLikeModel(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim):
        """
            num_embeddings: количество слов, для которых обучаем эмбеддинги
            embedding_dim: размерность эмбеддинга
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(num_embeddings+1, embedding_dim)
        
    def forward(self, ids, offsets):
        """
            ids: вытянутая посл-ть индексов слов вопросов, попавших в батч
            offsets: сдвиги для вопросов, попавших в батч
            
            returns: векторы вопросов
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        embedded_ids = self.embedding(ids)
        question_vectors = []
        offset_prev = 0
        for offset in offsets:
            if offset - offset_prev == 0:
                question_vectors.append(self.embedding(torch.tensor(self.num_embeddings).to(device)))
            else:
                question_vectors.append(embedded_ids[offset_prev:offset].mean(dim=0))
            offset_prev = offset
        question_vectors = torch.stack(question_vectors)
        return question_vectors

Создание модели

In [29]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
model_dssm = DssmLikeModel(len(train_preproc_vocab), 300)

**Критерий оптимизации** для NTExentLoss выглядит как:

$$\mathcal{L}(Q, D) = -0.5 \log diag(softmax(QD^T / \alpha)) - 0.5 \log diag(softmax(DQ^T / \alpha)),$$

где:
* $Q \in \mathbb{R}^{b \times d}$ - эмбеддинги вопросов, 
* $D \in \mathbb{R}^{b \times d}$ - эмбеддинги соответствующих вопросам дубликатов,
* $b$ - количество пар (вопрос, дубликат), $d$ - размерность эмбеддингов, $\alpha$ - гиперпараметр лосса. 
* Softmax берется по рядам
* Матрицы $Q, D$ содержат нормированные эмбеддинги, т.е. считается именно косинус.

In [30]:
class NTExentLoss(nn.Module):
    
    def __init__(self, alpha=1., eps=1e-8):
        """
            alpha: коэффициент, на который мы делим скоры перед софтмаксом
            eps: ||v|| = min(eps, ||v||)
        """
        super().__init__()
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.alpha = alpha
        self.eps = eps
        
    def _normalize(self, embeddings):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            
            returns: матрица такого же размера, но с нормироваными векторами
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        norm = embeddings.norm(dim=1, keepdim=True)
        return embeddings / (norm + self.eps)
    
    def forward(self, embeddings, positives):
        """
            embeddings: матрица размера [batch_size, embedding_dim]
            positives: матрица такого же размера, с позитивами для векторов из матрицы embeddings
            
            returns: NT-Exent loss
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        embeddings_norm, positives_norm = self._normalize(embeddings), self._normalize(positives)
        scores = embeddings_norm @ positives_norm.T / self.alpha
        loss = -0.5 * (torch.log_softmax(scores, dim=1).diag() + torch.log_softmax(scores.T, dim=1).diag())
        return loss.mean()

**Создайте пайплайн** для обучения и валидации, используя предложенный шаблон.

Залогируйте с помощью **torch.utils.tensorboard.SummaryWriter** две величины:
1. Лосс для каждого батча
2. Лосс на валидации для каждой эпохи

In [31]:
import time
import shutil
import os
import torch

from torch.utils.tensorboard import SummaryWriter
    
    
class Trainer:
    
    def __init__(
            self, 
            model, 
            optimizer, 
            criterion, 
            logdir=None, 
            device=None
    ):
        """
            model: объект класса DssmModel
            optimizer: оптимизатор
            criterion: критерий оптимизации
            logdir: директория, в которую SummaryWriter должен писать логи
            device: девайс (cpu или cuda), на котором надо производить вычисления
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.logdir = logdir
        self.device = device

        self._writer = SummaryWriter(log_dir=logdir)
        self.model.to(self.device)
    
    def _calculate_loss(self, batch):
        """
            batch: батч из индексов и сдвигов для вопросов и их дубликатов
            
            returns: посчитанный для батча лосс
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        questions, duplicates = batch
        questions_embed = self.model(questions[0].to(self.device), questions[1])
        duplicates_embed = self.model(duplicates[0].to(self.device), duplicates[1])
        return self.criterion(questions_embed, duplicates_embed)
    
    def _train_step(self, dataloader):
        """
            dataloader: даталоадер для обучения
            
            returns: лосс на датасете для обучения
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model.train()
        total_loss = 0
        for batch in dataloader:
            self.optimizer.zero_grad()
            loss = self._calculate_loss(batch)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        if self._writer is not None:
            self._writer.add_scalar('train/loss', avg_loss, global_step=self._n_epoch)
        return avg_loss
    
    def _eval_step(self, dataloader):
        """
            dataloader: даталоадер для валидации
            
            returns: лосс на валидации
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for batch in dataloader:
                loss = self._calculate_loss(batch)
                total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        return avg_loss
    
    def train(self, dataloaders, n_epochs, verbose=False):
        """
            dataloaders: словарь вида {'train': train_dataloader, 'eval': eval_dataloader}
            n_epochs: количество эпох обучения
            verbose: нужно ли выводить каждую эпоху информацию про лоссы
        """
        start = time.time()
        self._n_epoch = 0
        for epoch in range(n_epochs):
            train_loss = self._train_step(dataloaders['train'])
            
            eval_loss = self._eval_step(dataloaders['eval'])
            if self._writer is not None:
                self._writer.add_scalar('eval/loss', eval_loss, global_step=self._n_epoch)
                
            if verbose:
                print(
                    'epoch: {:>2}, train loss: {:.4f}, eval loss: {:.4f}, time: {:.4f}' \
                        .format(epoch + 1, train_loss, eval_loss, time.time() - start)
                )
                    
            self._n_epoch += 1

Предлагается использовать для оптимизации Адам и обучать модель 10-60 эпох.

Для этой части задания GPU даёт существенное ускорение при обучении, поэтому стоит по возможности делать обучение с большим batch size'ом и на GPU.

In [32]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
train_dataloader = DataLoader(train_train, batch_size=1024, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_train, batch_size=1024, shuffle=True, collate_fn=collate_fn)

In [33]:
model_dssm = DssmLikeModel(len(train_preproc_vocab), 300)

optimizer = optimizer = torch.optim.Adam(model_dssm.parameters(), lr=3e-4)
criterion = NTExentLoss()
device="cuda"
trainer = Trainer(model_dssm, optimizer, criterion, logdir="logs", device=device)

In [35]:
dataloaders = {'train': train_dataloader, 'eval': val_dataloader}
trainer.train(dataloaders, n_epochs=100, verbose=True)

KeyboardInterrupt: 

Модель обучил, логи стёрлись... печаль.. обучалось 75 эпох до трейн лосса где-то около 6.19

Осталось достать из модели обученные под задачу векторы слов, составить маппинг слов в векторы, создать **Embedder** и **Scorer** и провалидировать качество на нашей исходной валидации, которой мы пользовались в первых двух частях.

Чтобы достать из модели веса, можно использовать `model._embeddings.weight.cpu().detach().numpy()`

In [39]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################
embeddings = trainer.model.embedding.weight.cpu().detach().numpy()

In [41]:
word2vec_embeddings = {}
for word, idx in train_preproc_vocab.items():
    word2vec_embeddings[word] = embeddings[idx]

In [45]:
for sample in tqdm.tqdm(val_preproc):
    for text in sample:
        for word in text.split():
            if word not in word2vec_embeddings:
                word2vec_embeddings[word] = embeddings[-1]

100%|██████████| 3760/3760 [00:02<00:00, 1635.63it/s]


In [46]:
len(train_preproc_vocab), len(word2vec_embeddings)

(131030, 133192)

In [47]:
embedder = Embedder(word2vec_embeddings, dim=300)

scorer = Scorer(
    k=[1, 5, 10, 100, 500, 1000],
    embedder=embedder
)

hits = scorer(val_preproc, verbose=True)

100%|██████████| 3760/3760 [11:43<00:00,  5.35it/s]


In [48]:
hits

{1: 0.39654255319148934,
 5: 0.5888297872340426,
 10: 0.6696808510638298,
 100: 0.8981382978723405,
 500: 0.9848404255319149,
 1000: 1.0}

Одним из критериев получения полных баллов является значение метрики **hits@500** $\geqslant 0.98$.

## Дополнительная часть (до 3 баллов)

Каждый из пунктов при успешном выполнении гарантирует как минимум один дополнительный балл. Максимум вам будет зачтено три пункта. 
1. Обучить триграммную модель на косинусную близость вместо евклидового расстояния, получить прирост качества (относительно триграммной модели с MSE)
2. Обучить в качестве триграммной модели char-biLSTM вместо мешка векторов триграмм, получить прирост качества (относительно триграммной модели с таким же критерием оптимизации)
3. Усложнить модель в части 3, добавить к мешку вектора словесных униграмм также мешок векторов словесных биграмм и мешок векторов буквенных триграмм (сделать модель более похожей на настоящий dssm), получить прирост качества
4. Модифицировать модель в части 3 произвольным образом (добавить MLP, нормализации, дропаут, сделать bilstm поверх последовательности векторов слов, трансформер и т.д.), получить прирост качества
5. Сделать модель с ранним связыванием (early fusion) - векторы вопросов конкатенируются и проходят через MLP (с возможными модификациями) перед созданием предсказания. Hint: возможно стоит предобучить эмбеддинги слов с помощью NT-Exent перед обучением финальной модели.